# Example 2: 

## Thermal model

### Isothermal:
temperature stays constant.
### Lumped: 
the temperature is taken to be homogeneous in the battery, so only the average temperature is computed.
### X-lumped: 
the temperature is taken to be homogeneous across the thickness of the cell, but can vary in the directions parallel to the current collectors. Need to be used in conjunction with a current collector model.
### X-full: 
the temperature is allowed to vary across the thickness of the cell.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pybamm

 Lets load up DFN model and DFN model with a lumped thermal model.
  
We can do this by passing a dictionary of options to the model when we create it. We can take a look at the documentation to see the available model options.
 

In [3]:
DFN = pybamm.lithium_ion.DFN(name = "DFN model")

TDFN = pybamm.lithium_ion.DFN(
       options = {"thermal":"lumped"},
       name = "Lumped+DFN model")

In [4]:
models = [DFN, TDFN]

sols =[]

for model in models:
    sim = pybamm.Simulation(model)
    sol = sim.solve([0,3600])
    sols.append(sol)

In [5]:
pybamm.dynamic_plot(sols, ["Terminal voltage [V]", "Volume-averaged cell temperature [K]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [6]:
experiment = pybamm.Experiment(
    [
        ( 
               "Discharge at 1C until 3.7V",
               "Charge at 0.3C for 3600 seconds"
        )
    ]*2,
)

In [7]:
sols =[]

for model in models:
    sim = pybamm.Simulation(model, experiment = experiment)
    sol = sim.solve()
    sols.append(sol)

pybamm.dynamic_plot(sols, ["Terminal voltage [V]", "Volume-averaged cell temperature [K]"])    

interactive(children=(FloatSlider(value=0.0, description='t', max=2.444046187137165, step=0.024440461871371652…

# Exercise: 

Compare the results of the DFN model with isothermal and x-full. Plot the relevant variables, and recall that you can find the list of variables to plot by typing model.variable_names(). What differences do you observe in the models?



In [8]:
thermal_options = ["isothermal", "x-full"]
sols = []
for option in thermal_options:
    model = pybamm.lithium_ion.DFN(name = option, options={"thermal":option})
    simulation = pybamm.Simulation(model)
    sols.append(simulation.solve([0,3600]))


In [9]:
pybamm.dynamic_plot(
    sols,
    output_variables=[
        "Negative particle surface concentration [mol.m-3]",
        "Electrolyte concentration [mol.m-3]",
        "Positive particle surface concentration [mol.m-3]",
        "Negative electrode potential [V]",
        "Electrolyte potential [V]",
        "Positive electrode potential [V]",
        "Current [A]",
        "Voltage [V]",
        "Cell temperature [K]",
    ],
)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

We observe that the temperature in the isothermal model remains constant, while for the x-full model the temperature rises almost 20 K. Despite being a spatial-dependent model, the temperature gradients are negligible, which is reasonable given how thin the cell is. We also observe that the voltage for the isothermal case is lower than for the temperature dependent one, given that at higher temperatures the overpotentials are smaller.

Reference: 
The mechanical model is from the paper "Electrochemical Thermal-Mechanical Modelling of Stress Inhomogeneity in Lithium-Ion Pouch Cells", Ai et al, 2020.